Sentiment analysis on Instagram comments

In [63]:
!pip install emosent-py


[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
# nltk.download('vader_lexicon')
# nltk.download('stopwords')
# nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\viraj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\viraj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
#essential libraries
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

from googletrans import Translator



In [3]:
data= pd.read_csv('cleaned_data.csv')
data.head(2)
df=data.copy()

In [4]:
df.dropna(inplace=True)

In [5]:
translator = Translator()

def preprocess_text_and_extract_emojis(text):
    try:
        translated = translator.translate(text, 'en')
        en_text = translated.text
        print("translated:",en_text)
        
        en_text = re.sub(r'@\S+', '', en_text)  
        en_text = re.sub(r'\W', ' ', en_text)  
        en_text = re.sub(r'\d', ' ', en_text)  
        en_text = en_text.lower()  
        
        tokens = word_tokenize(en_text)
        tokens = [w for w in tokens if w not in stopwords.words('english')]
        cleaned_text = ' '.join(tokens)
        print("clean:",cleaned_text)
        return cleaned_text
    
    except Exception as e:
        print(f"Error processing text '{text}': {e}")
        return ""
    
df['cleaned_text']=df['Text'].apply(preprocess_text_and_extract_emojis)


translated: #elonbuywatercoin @the_bentrepreneur @water.solana
clean: elonbuywatercoin
translated: Man those are ugly 😂
clean: man ugly
translated: Vacuum Cleaner
clean: vacuum cleaner
translated: 🔥🙌
clean: 
translated: 😍😍
clean: 
translated: @Benyamin_co.iR app
clean: app
translated: I mean, some people like this pick up. Some don't, but everybody knows that the 1980 Toyota Pickup is the role model for very pickup after 1980. It is the undestructable and most relaiable Pick Up ever made. Even you, dear reader, know it.
clean: mean people like pick everybody knows toyota pickup role model pickup undestructable relaiable pick ever made even dear reader know
translated: These things look unbelievably stupid on the roads!
clean: things look unbelievably stupid roads
translated: @nemet_amanda these are the cars I tried to explain but couldn’t the other day…
clean: cars tried explain day
translated: @jan_michaud that’s literally a robot! What the ?! 😳
clean: literally robot
translated: @kla

In [45]:
df

,Unnamed: 0,User,Text,Timestamp,cleaned_text,Positive,Negative,Neutral
0,0,dambachtale_schawale,#elonbuywatercoin @the_bentrepreneur @water.so...,2024-07-10 12:03:26+00:00,elonbuywatercoin,0.000,0.000,1.000
1,1,lifewiththefishers,Man those are ugly 😂,2024-07-11 02:52:05+00:00,man ugly,0.000,0.767,0.233
2,2,hamidamjadiii,جاروبرقی 😂,2024-07-11 10:27:38+00:00,vacuum cleaner,0.630,0.000,0.370
3,3,tyoncybertruck21,🔥🙌,2024-07-12 03:57:56+00:00,,0.000,0.000,0.000
4,4,ariad.ff4,😍😍,2024-07-12 06:33:25+00:00,,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...
847,847,ScienceProjectSuccessHighSchool,Collaborating on a science project that receiv...,2017-08-18 18:20:00,collaborating science project received recogni...,0.545,0.000,0.455
848,848,BirthdayPartyJoyHighSchool,Attending a surprise birthday party organized ...,2018-06-22 14:15:00,attending surprise birthday party organized fr...,0.784,0.000,0.216
849,849,CharityFundraisingTriumphHighSchool,Successfully fundraising for a school charity ...,2019-04-05 17:30:00,successfully fundraising school charity initia...,0.709,0.000,0.291
850,850,MulticulturalFestivalJoyHighSchool,"Participating in a multicultural festival, cel...",2020-02-29 20:45:00,participating multicultural festival celebrati...,0.639,0.000,0.361


In [6]:
sen= SentimentIntensityAnalyzer()

In [7]:

df["Positive"]=[sen.polarity_scores(i)['pos'] for i in df['cleaned_text']]
df["Negative"]=[sen.polarity_scores(i)['neg'] for i in df['cleaned_text']]
df["Neutral"]=[sen.polarity_scores(i)['neu'] for i in df['cleaned_text']]


In [8]:

def senti(t):
    pos=t['Positive']
    neg=t['Negative']
    neu=t['Neutral']
    if pos>neg and pos>neu:
        return "Positive"
    elif neg>pos and neg>neu:
        return "Negative"
    elif neu>pos and neu>neg:
        return "Neutral"
    else:
        return "Neutral"
    
df["Sentiment"]= df.apply(senti,axis=1)

In [9]:
df

,Unnamed: 0,User,Text,Timestamp,cleaned_text,Positive,Negative,Neutral,Sentiment
0,0,dambachtale_schawale,#elonbuywatercoin @the_bentrepreneur @water.so...,2024-07-10 12:03:26+00:00,elonbuywatercoin,0.000,0.000,1.000,Neutral
1,1,lifewiththefishers,Man those are ugly 😂,2024-07-11 02:52:05+00:00,man ugly,0.000,0.767,0.233,Negative
2,2,hamidamjadiii,جاروبرقی 😂,2024-07-11 10:27:38+00:00,vacuum cleaner,0.630,0.000,0.370,Positive
3,3,tyoncybertruck21,🔥🙌,2024-07-12 03:57:56+00:00,,0.000,0.000,0.000,Neutral
4,4,ariad.ff4,😍😍,2024-07-12 06:33:25+00:00,,0.000,0.000,0.000,Neutral
...,...,...,...,...,...,...,...,...,...
847,847,ScienceProjectSuccessHighSchool,Collaborating on a science project that receiv...,2017-08-18 18:20:00,collaborating science project received recogni...,0.545,0.000,0.455,Positive
848,848,BirthdayPartyJoyHighSchool,Attending a surprise birthday party organized ...,2018-06-22 14:15:00,attending surprise birthday party organized fr...,0.784,0.000,0.216,Positive
849,849,CharityFundraisingTriumphHighSchool,Successfully fundraising for a school charity ...,2019-04-05 17:30:00,,0.000,0.000,0.000,Neutral
850,850,MulticulturalFestivalJoyHighSchool,"Participating in a multicultural festival, cel...",2020-02-29 20:45:00,participating multicultural festival celebrati...,0.639,0.000,0.361,Positive


In [10]:
df.dropna(inplace=True)

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib


In [24]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import joblib

# Load your data
# df = pd.read_csv('your_data.csv')  # Load your data here

# Separate features and target
X = df['cleaned_text']
y = df['Sentiment']

# Create a pipeline with vectorizer, SMOTE, and classifier
pipeline = ImbPipeline([
    ('vectorizer', TfidfVectorizer()),  # Vectorization step
    ('smote', SMOTE()),  # SMOTE step
    ('clf', LogisticRegression(class_weight='balanced'))  # Classifier step
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
pipeline.fit(X_train, y_train)

# Save the pipeline
joblib.dump(pipeline, 'sentiment_model.pkl')


['sentiment_model.pkl']

In [25]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       0.64      0.50      0.56        14
     Neutral       0.80      0.88      0.84       111
    Positive       0.72      0.58      0.64        45

    accuracy                           0.77       170
   macro avg       0.72      0.65      0.68       170
weighted avg       0.76      0.77      0.76       170

